In [16]:
import numpy as np

import pandas as pd

# used to grab the stock prices, with yahoo

import pandas_datareader as web

from datetime import datetime

# to visualize the results

import matplotlib.pyplot as plt

import seaborn

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import os
import re
from time import sleep


In [18]:
# load stock screener data from nasdaq

chromeOptions = webdriver.ChromeOptions()
cur_dir = os.getcwd()
prefs = {"download.default_directory" : cur_dir}
chromeOptions.add_experimental_option("prefs",prefs)

driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), chrome_options=chromeOptions)
driver.get("https://www.nasdaq.com/market-activity/stocks/screener")

driver.implicitly_wait(5)
link = driver.find_element_by_class_name("nasdaq-screener__download")
link.click()


file_location = os.path.join(cur_dir, 'nasdaq_screener.csv')

if os.path.isfile(file_location):
    os.remove(file_location) # delete if previous file exists

# delete suffix from the csv file's name
file_regex = re.compile('nasdaq_screener_\d+\.csv')

sleep(15)

while not os.path.isfile(file_location):
    for root, dirs, files in os.walk(cur_dir):
        for f in files:
            if file_regex.match(f):
                os.rename(f, 'nasdaq_screener.csv')
                break


driver.close()



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/nadavsmacbookpro/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  if sys.path[0] == '':


In [7]:
# get tickers from csv
stocks_screener = pd.read_csv('nasdaq_screener.csv')
stocks_screener = stocks_screener.sort_values(by='Market Cap', ascending=False)

FileNotFoundError: [Errno 2] No such file or directory: 'nasdaq_screener.csv'

In [8]:
seaborn.set(rc={"figure.dpi":300, 'savefig.dpi':300})

# select start date for correlation window as well as list of tickers

start = datetime(2021, 1, 1)

symbols_list = list(stocks_screener['Symbol'][:50])
symbols=[]

NameError: name 'stocks_screener' is not defined

In [ ]:
#pull price using iex for each symbol in list defined above

for ticker in symbols_list:

    r = web.DataReader(ticker, 'yahoo', start)

    # add a symbol column

    r['Symbol'] = ticker

    symbols.append(r)

In [ ]:
# concatenate into df

df = pd.concat(symbols)

df = df.reset_index()

df = df[['Date', 'Close', 'Symbol']]

df.head()

df_pivot = df.pivot('Date', 'Symbol', 'Close').reset_index()

df_pivot.head()

In [ ]:
corr_df = df_pivot.corr(method='pearson')

# reset symbol as index (rather than 0-X)

corr_df.head().reset_index()

#del corr_df.index.name

corr_df.head(10)
# corr_df = corr_df.reindex(corr_df.mean().sort_values().index, axis = 0)

In [ ]:
# take the bottom triangle since it repeats itself

mask = np.zeros_like(corr_df)

mask[np.triu_indices_from(mask)] = True

# generate plot
seaborn.set(font_scale=0.4)

seaborn.heatmap(corr_df, cmap='RdYlGn', vmax=1.0, vmin=-1.0, mask=mask, linewidths=2.5)

plt.yticks(rotation=0)

plt.xticks(rotation=90)
plt.savefig("1.png")

In [ ]:
corr_df["Mean"] = corr_df.mean(axis=0)
corr_df = corr_df.sort_values(by=['Mean'])
# corr_df = corr_df.drop(columns=['Mean'])
corr_df

In [ ]:
thresholds = [-0.5, 0, 0.5]
a = list(corr_df.where(corr_df < thresholds[0]).stack().index)
b = list(corr_df.where((thresholds[0] <= corr_df) &  (corr_df < thresholds[1])).stack().index)
c = list(corr_df.where((thresholds[1] <= corr_df) &  (corr_df < thresholds[2])).stack().index)
d = list(corr_df.where((thresholds[2] <= corr_df ) & (corr_df  < 1)).stack().index)
d



In [ ]:
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

sse = []
for k in range(2, 15):
    kmeans = KMeans(n_clusters=k).fit(corr_df)
    sse.append(kmeans.inertia_)
plt.plot(range(2, 15), sse)
# plt.show()
plt.savefig("2.png")


In [ ]:
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters).fit(corr_df)
centroids = kmeans.cluster_centers_
X = corr_df.to_numpy()
colors = ['purple', 'blue', 'green', 'orange', 'red']
# plt.scatter(X[:,0],X[:,1], c = kmeans.labels_, cmap =colors)
#
# plt.show()

ax = sns.scatterplot(X[:, 0], X[:, 1], hue=kmeans.labels_, palette=colors, alpha=0.5, s=7)
ax = sns.scatterplot(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
                     hue=range(num_clusters), palette=colors, s=20, legend=False, ax=ax)
plt.show()

In [ ]:
Company = pd.DataFrame(corr_df.index)
cluster_labels = pd.DataFrame(kmeans.labels_)
df = pd.concat([Company, cluster_labels],axis = 1)
df['color'] = [colors[label] for label in kmeans.labels_]
df